In [1]:
##############
#Dependencies#
##############
%matplotlib notebook

import pandas
import gmaps
import numpy as np
import folium
from config import (gkey, 
#                     route)
                   )

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
route = ['S567','S583','S568','S582','S569','S570','S581',
'S571','S579','S572','S578','S573',"S577","S587"]
station_df = pandas.read_csv('data/station_config.csv')
route_df= pandas.read_csv('data/route_summary.csv')
station_df = station_df.set_index("Station")
route_df = route_df.set_index("Station")
route_df= route_df.replace("UNKNOWN",0)
route_df= route_df.fillna(0)
route_df.head()

,Station.1,Time,Occupancy,Speed,Flow
Station,,,,,
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Wed Oct 03 01:16:59 CDT 2018,0.31,54,0
S567,0.0,Sun Oct 07 00:22:29 CDT 2018,3.83,62,560
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0
S583,0.0,Wed Oct 03 01:16:59 CDT 2018,0.24,55,0


In [4]:

Results = {}


locations = []

for station in route:
        Flow =  int(route_df.loc[station,'Flow'].tail(1)[0])
        Speed = int(route_df.loc[station,'Speed'].tail(1)[0])

        Max_Flow = route_df.loc[station,'Flow'].astype(dtype="int")
        Max_Flow = max(Max_Flow.values)
        Max_Speed = route_df.loc[station,'Speed'].astype(dtype="int")
        Max_Speed = max(Max_Speed.values)
        
        lat, lon = station_df.loc[station, ['Lat', 'Lng']]
        locations.append((lat, lon))
    
        Results[station] = {
            "Current Speed" : Speed,
            "Current Flow" : Flow,
            "Max Speed" : Max_Speed,
            "Max Flow" : Max_Flow,
            "% of Max Speed" : round(Speed / Max_Speed * 100, 2),
            "% of Max Flow" : round(Flow / Max_Flow * 100, 2) ,
            "Lat":lat, 
            "Lon":lon}

In [5]:
Results = pandas.DataFrame(Results).T
Results.head()
Results

,% of Max Flow,% of Max Speed,Current Flow,Current Speed,Lat,Lon,Max Flow,Max Speed
S567,21.43,100.00,120.0,81.0,44.97447,-93.24870,560.0,81.0
S583,100.00,100.00,450.0,67.0,44.98175,-93.24397,450.0,67.0
S568,0.00,0.00,0.0,0.0,44.98399,-93.24197,480.0,48.0
S582,100.00,100.00,280.0,89.0,44.99115,-93.23659,280.0,89.0
S569,36.36,89.25,120.0,83.0,44.99116,-93.23629,330.0,93.0
S570,27.27,100.00,120.0,87.0,44.99644,-93.23823,440.0,87.0
S581,23.53,100.00,160.0,76.0,44.99646,-93.23853,680.0,76.0
S571,26.67,100.00,160.0,87.0,45.00199,-93.23465,600.0,87.0
S579,46.15,100.00,240.0,68.0,45.00091,-93.22346,520.0,68.0
S572,66.67,90.79,80.0,69.0,45.00070,-93.22349,120.0,76.0


In [6]:
Results['Lat']['S573']

45.00515

In [33]:
folium_map = folium.Map(locations[0],
                        zoom_start=13,
                        tiles="CartoDB positron")
for index, row in Results.iterrows():

    
    folium.CircleMarker(location=(row["Lat"],
                                  row["Lon"]),
                        radius=1,
                        
                        fill=True).add_to(folium_map)

In [38]:
Features = []
Last_Sensor = []
for index, row in Results.iterrows():
  
    print(Last_Sensor)
    Current_Sensor = (row['Lat'], row['Lon'])
    print(Current_Sensor)
    if Last_Sensor == [] :
        Last_Sensor = (row['Lat'], row['Lon'])
    else:
    
        folium.PolyLine([Current_Sensor,Last_Sensor],
         weight=row['Current Flow'] / 30,
                        popup=f"Speed:{row['Current Speed']} Flow: {row['Current Flow']}").add_to(folium_map)

        Last_Sensor = (row['Lat'], row['Lon'])





[]
(44.974470000000004, -93.2487)
(44.974470000000004, -93.2487)
(44.98175, -93.24396999999999)
(44.98175, -93.24396999999999)
(44.983990000000006, -93.24197)
(44.983990000000006, -93.24197)
(44.99115, -93.23659)
(44.99115, -93.23659)
(44.99116, -93.23629)
(44.99116, -93.23629)
(44.99644, -93.23823)
(44.99644, -93.23823)
(44.99646, -93.23853000000001)
(44.99646, -93.23853000000001)
(45.00199, -93.23465)
(45.00199, -93.23465)
(45.00091, -93.22346)
(45.00091, -93.22346)
(45.0007, -93.22349)
(45.0007, -93.22349)
(45.00302, -93.21486999999999)
(45.00302, -93.21486999999999)
(45.00515, -93.20947)
(45.00515, -93.20947)
(45.01014, -93.2004)
(45.01014, -93.2004)
(45.00891, -93.20233)


In [39]:
folium_map
